In [38]:
from pprint import pprint
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

import tensorflow as tf
import edward as ed
from edward.models import Bernoulli, Categorical, Normal, Empirical, Multinomial

from utils.utils import load_dataframe, preprocess, transition_matrix

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)
sns.set_style('whitegrid')

## Data

In [3]:
df = load_dataframe()

Loading raw data from hdf5 cache...
Fetching raw data took 2.85 seconds
id             int64   
loan_status    category
age_of_loan    int64   
term           category
dtype: object
Retrieved 40,268,594 rows, 4 columns


In [4]:
# note I made this filter on term == 36 and age_of_loan <= 36 
df = preprocess(df)

Mapping transformations...
Preprocessing data...
Caching...
Preprocessing and caching took 129.73 seconds
id             int64
loan_status    int64
age_of_loan    int64
term           int64
dtype: object
Preprocessed 27,641,460 rows, 4 columns


In [9]:
realized_transitions = transition_matrix(df)

Loading transition matrix from hdf5 cache...
Fetching transition matrix took 0.04 seconds


In [10]:
realized_transitions

loan_status_y,Charged Off,Current,Default,Fully Paid,In Grace Period,Issued,Late (16-30 days),Late (31-120 days)
loan_status_x,,,,,,,,
Charged Off,0,0,0,0,0,0,0,0
Current,774,24453702,3,707322,5831,0,160366,62102
Default,28897,147,2297,71,0,0,4,506
Fully Paid,0,0,0,8063,12,0,101,72
In Grace Period,0,276,0,11,22,0,59,41
Issued,0,17206,0,670,1,0,38,1
Late (16-30 days),4548,32376,0,2066,257,0,13413,119621
Late (31-120 days),105934,25434,29802,2146,56,0,3292,332762


## Model

In [13]:
# from issue https://github.com/blei-lab/edward/issues/450
chain_len = 36
n_states = df.loan_status.unique().shape[0]

# create default starting state probability vector
x_0 = Categorical(probs=tf.fill([n_states], 1.0 / n_states))

# transition matrix
T = tf.nn.softmax(tf.Variable(tf.random_uniform([n_states, n_states])), axis=0)

# MODEL
x = []
for _ in range(chain_len):
    x_tm1 = x[-1] if x else x_0
    x_t = Categorical(probs=T[:, x_tm1])
    x.append(x_t)

## Inference

In [20]:
df.dtypes

id                int64
loan_status       int64
age_of_loan       int64
term              int64
previous_month    int64
dtype: object

In [36]:
df.loc[df.id == 55742].loan_status.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3], dtype=int64)

In [39]:
qx = [Categorical(probs=tf.nn.softmax(tf.Variable(tf.ones(n_states)))) for _ in range(chain_len)]

x_data = df.loc[df.id == 55742].loan_status.values

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(T))

    inference = ed.KLqp(dict(zip(x, qx)), dict(zip(x, x_data)))
    inference.run(n_iter=20000)

    print(sess.run(T))
    print('qx:')
    pprint(sess.run([foo.probs for foo in qx]))
    print('x:')
    pprint(sess.run([foo.probs for foo in x]))

[[0.16768937 0.15017682 0.18834004 0.12487649 0.1369074  0.1390993
  0.09843953 0.17314914]
 [0.11174133 0.20890342 0.0896683  0.1003598  0.11244719 0.11988048
  0.1185196  0.09314825]
 [0.13615161 0.08183575 0.16915742 0.1218937  0.18143697 0.12976703
  0.17464876 0.20318094]
 [0.076153   0.1444956  0.10090057 0.14442232 0.11279707 0.12880106
  0.17662922 0.11903022]
 [0.11432905 0.1446221  0.10026382 0.09979181 0.13151404 0.11792227
  0.13132149 0.11179972]
 [0.15366088 0.08062167 0.13380732 0.18301295 0.0898024  0.10026496
  0.12206826 0.08137009]
 [0.09872429 0.10035351 0.1300261  0.12357572 0.10371384 0.16668496
  0.0896854  0.09806894]
 [0.14155048 0.08899108 0.0878364  0.10206727 0.1313811  0.09757988
  0.08868779 0.12025262]]
20000/20000 [100%] ██████████████████████████████ Elapsed: 133s | Loss: 0.815
[[0.0892266  0.09454423 0.09510345 0.09291577 0.09636167 0.09006771
  0.09058608 0.08932319]
 [0.11863378 0.1162275  0.12270942 0.12013933 0.11939491 0.12388707
  0.12711583 0.12